# Data Curation Notebook A - Processing of GWAS and RV Data

This notebook performs the inital processing of common and rare variant association data:

- Filtering CV and RV studies
- Filtering gene associations
- Converting to NCBI Gene IDs
- Cleaning the trait EFO codes

**Inputs:**
* GWAS Study Information (`gwas-catalog-v1.0.3.1-studies-r2025-03-26.tsv.gz`)
* GWAS Associations (`gwas_catalog_Jan29_2025.txt.gz`)
* GWAS Trait EFO Mappings (`gwas_catalog_trait-mappings_r2025-03-26.tsv.gz`)
* RV Study Information (`rv_study_info_cleaned_with_manual.tsv`)
* RV Associations (`rv_study_info_cleaned_with_manual_mapped_Mar28.tsv`)

**Figures generated in this notebook:**
- None

## Set Up

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from neteval import gene_mapper as gm
from neteval import query_ensembl as qe
from neteval import query_hgnc as qh
import re
from sklearn.metrics.pairwise import cosine_similarity
import sentence_transformers
model = sentence_transformers.SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
cwd = os.getcwd()
datadir = os.path.join(cwd, '..')

## GWAS Catalog - Common Variant Associations

GWAS catalog study information can be downloaded from: https://www.ebi.ac.uk/gwas/docs/file-downloads (All studies v1.0.3.1)

### Filter Studies

GWAS studies are filtered to:
* Remove studies with background traits
* Keep only genome-wide studies
* Exclude studies with missing trait information, or multiple trait mappings
* Exclude studies with too few associations

In [5]:
# Get study information
study_info = pd.read_csv(os.path.join(datadir,'Reference_Data', 'gwas-catalog-v1.0.3.1-studies-r2025-03-26.tsv.gz'), 
                         sep='\t', low_memory=False)
print(f'Initial Studies:', len(study_info))

# Remove studies with background traits
study_info = study_info[study_info['MAPPED BACKGROUND TRAIT'].isna()]
print(f'- Studies with background trait:', len(study_info))

# Use only genome-wide studies
study_info['GENOTYPING TECHNOLOGY'] = study_info['GENOTYPING TECHNOLOGY'].apply(lambda x: x.split('[')[0].strip())
study_info = study_info[study_info['GENOTYPING TECHNOLOGY'].isin(['Genome-wide genotyping array', 
       'Genome-wide genotyping array, Genome-wide sequencing',])]
print(f'- Not genome-wide studies:', len(study_info))

# Remove studies with missing trait information
study_info = study_info.dropna(subset=['DISEASE/TRAIT', 'MAPPED_TRAIT', 'MAPPED_TRAIT_URI'])
print(f'- Studies with missing trait information:', len(study_info))

# Remove studies mapped to multiple traits
study_info = study_info[~study_info['MAPPED_TRAIT_URI'].str.contains(', ')]
print(f'- Studies with multiple traits:', len(study_info))

# Keep studies with at least 3 associations (will be filtered based on genes later)
study_info = study_info[study_info['ASSOCIATION COUNT'] >= 3]
print(f'- Studies with at < 3 associations:', len(study_info))
print(f'Total filtered studies:', len(study_info))

Initial Studies: 138810
- Studies with background trait: 124463
- Not genome-wide studies: 101460
- Studies with missing trait information: 101402
- Studies with multiple traits: 95859
- Studies with at < 3 associations: 19452
Total filtered studies: 19452


### Clean trait information

Mappings between trait descriptions and EFO terms are in the GWAS Catalog are not one-to-one. Therefore, we will take the best matched EFO term for each unique trait description.   

Trait mappings can be downloaded from: https://www.ebi.ac.uk/gwas/docs/file-downloads (GWAS to EFO mappings)

In [21]:
trait_info = pd.read_csv(os.path.join(datadir,'Reference_Data',  'gwas_catalog_trait-mappings_r2025-03-26.tsv.gz'), sep='\t')

# Identify traits present in filtered studies, and the number of occurences of each trait
trait_info = trait_info[trait_info['Disease trait'].isin(study_info['DISEASE/TRAIT'])]
trait_disease_counts = trait_info['Disease trait'].value_counts()

In [7]:
# Identify one-to-one mappings
one_match = trait_disease_counts[trait_disease_counts == 1].index
one_match = trait_info[trait_info['Disease trait'].isin(one_match)]

In [8]:
# Initiate dictionary of trait mappings
match_dict = {}
for i, row in one_match.iterrows():
    match_dict[row['Disease trait']] = (row['EFO term'], row['EFO URI'])

### Find best matches for traits with multiple mappings


In [9]:
multi_match = trait_disease_counts[trait_disease_counts > 2].index
multi_match = trait_info[trait_info['Disease trait'].isin(multi_match)]

In [10]:
# Sentence embedding cannot always capture acronyms, so need to expand the most common ones
def replace_acronyms(text, acronym_dict):

    """
    Replace acronyms in a string with their expansions followed by the original acronym in parentheses.
    
    Args:
        text (str): The input string containing acronyms.
        acronym_dict (dict): A dictionary where keys are acronyms (str) and values are the expansions (str).
    
    Returns:
        str: The text with acronyms replaced by their expansion and the original acronym in parentheses.
    """
    # This regex matches words consisting of at least two uppercase letters.
    acronym_pattern = re.compile(r'[A-Z]{2,}')
    
    def replacer(match):
        word = match.group(0)
        if word in acronym_dict:
            # Return the expansion with the original acronym in parentheses.
            return f"{acronym_dict[word]} ({word})"
        else:
            return word
    
    return acronym_pattern.sub(replacer, text)

def extract_acronyms(strings):
    """
    Extract acronyms from a list of strings and count their occurrences.
    Acronyms are identified as words that consist entirely of uppercase letters.
    
    Args:
        strings (list of str): List of strings to search for acronyms.
    
    Returns:
        dict: A dictionary with acronyms as keys and their counts as values.
    """
    acronym_pattern = re.compile(r'\b[A-Z]{2,}\b')
    acronym_counts = {}
    for text in strings:
        found = acronym_pattern.findall(text)
        for acronym in found:
            acronym_counts[acronym] = acronym_counts.get(acronym, 0) + 1
    return acronym_counts

def map_trait(trait_str, trait_info, usecol='Disease trait'):
    options = trait_info[trait_info[usecol]==trait_str]
    # embeddings
    trait_embedding = model.encode([trait_str.lower().replace('levels', 'measurement')])
    option_embeddings = model.encode([x.lower() for x in options['EFO term'].values])
    # cosine similarity
    similarities = cosine_similarity(trait_embedding, option_embeddings)
    best_option = options.iloc[np.argmax(similarities)]
    return (best_option['EFO term'], best_option['EFO URI'])

In [11]:
# Definitions of common acronyms

common_acronyms = {
    'LDL': 'low density lipoprotein',
    'HDL': 'high density lipoprotein',
    'BMI': 'body mass index',
    'VLDL': 'very low density lipoprotein',
    'IL': 'interleukin',
    'IDL': 'intermediate density lipoprotein',
    'FVC': 'forced vital capacity',
    'FEV': 'forced expiratory volume',
    'COVID': 'coronavirus',
    'HIV': 'human immunodeficiency virus',
    'SARS': 'severe acute respiratory syndrome',
    'APOE': 'apolipoprotein E',
    'APOB': 'apolipoprotein B',
    'APOA': 'apolipoprotein A',
}

acronym_expansions = {
    'ldl': 'low density lipoprotein',
    'hdl': 'high density lipoprotein',
    "FAW3": 'omega-3 polyunsaturated fatty acid',
    "FAW6": 'omega-6 polyunsaturated fatty acid',
    "FEC": "Forced Expiratory volume",
    "FEV": "Forced Expiratory volume",
    "HIV": "Human Immunodeficiency Virus",
    "IV": "4",
    "LDH": "lumbar disc herniation",
    "NHDL": 'non High density ipoprotein',
    "NSAID": 'nonsteroidal anti inflammatory drug',
    "ACACE": "Acetoacetate",                                
    "ACE": "Acetate",             
    "ALA": "Alanine",                       
    "ALB": "Albumin",  
    "APOB": "Apolipoprotein B",  
    "APOC": "Apolipoprotein C",  
    "BMI": "Body Mass Index",   
    "BP": "Blood Pressure",   
    "CHOLA": "Cholesterol",    
    "CIT": "Citrate",      
    "CRP": "C-Reactive Protein",   
    "DHA": "Docosahexaenoic Acid",  
    "DISTRIB": "Distribution",     
    "EGFR": "Estimated Glomerular Filtration Rate",   
    "FRAC": "Fraction",                              
    "FVC": "Forced Vital Capacity",               
    "GLN": "Glutamine",                            
    "GLOL": "Glycerol",                          
    "GLY": "Glycine",                             
    "GP": "Glycoproteins",                        
    "HDL": "High Density Lipoprotein",                  
    "HDLC": "High Density Lipoprotein Cholesterol",     
    "HEEL": "Heel Bone Mineral Density",  
    "HEIGHT": "Height",                         
    "HIS": "Histidine",     
    "IDL": "Intermediate Density Lipoprotein",  
    "IGF": "Insulin like Growth Factor",  
    "III": "Type III",     
    "INS": "Insulin",  
    "LA": "Linoleic Acid",   
    "LDL": "Low Density Lipoprotein",
    "LDLC": "Low Density Lipoprotein Cholesterol",
    "LIGHT": "Light Scatter",
    "MUFA": "Monounsaturated Fatty Acids",
    "PC": "Phosphatidylcholine",          
    "PHE": "Phenylalanine",               
    "PUFA": "Polyunsaturated Fatty Acids",
    "PYR": "Pyruvate",                    
    "RBC": "Red Blood Cell Count",        
    "SCZD": "Schizophrenia",              
    "SHBG": "Sex Hormone-Binding Globulin",
    "SPHERED": "Spherical Diameter",       
    "SYST": "Systolic Blood Pressure",     
    "TOTCHO": "Total Cholines",            
    "TOTCHOL": "Total Cholesterol",        
    "TOTPG": "Total Phosphoglycerides",    
    "TSCORE": "T-Score",                   
    "TYR": "Tyrosine",                     
    "VAL": "Valine",                       
    "VIT": "Vitamin (unspecified)",      
    "VLDL": "Very Low Density Lipoprotein", 
    "VLDLPL": "Very low density lipoprotein Phospholipids",  
    "VLDLTG": "Very low density lipoprotein Triglycerides",  
    "VOL": "Volume",            
    "Whr": "Waist Hip Ratio", 
    "whr": "Waist Hip Ratio",
    "WHR": "Waist Hip Ratio",            
    "XL": "Extra Large",                 
    "XS": "Extra Small"                  
}

all_acronyms = {**common_acronyms, **acronym_expansions}

In [12]:
# expand the acronymns in both data sets
study_info['TraitExp'] =study_info['DISEASE/TRAIT'].apply(lambda x: replace_acronyms(x, all_acronyms))
study_info['MappedExp'] = study_info['MAPPED_TRAIT'].apply(lambda x: replace_acronyms(x, all_acronyms))
trait_info['TraitExp'] = trait_info['Disease trait'].apply(lambda x: replace_acronyms(x, all_acronyms))
trait_info['MappedExp'] = trait_info['EFO term'].apply(lambda x: replace_acronyms(x, all_acronyms))

In [13]:
# identify the best matches
for i, row in tqdm(multi_match.iterrows()):
    match_dict[row['Disease trait']] = map_trait(row['Disease trait'], trait_info)

1807it [06:41,  4.50it/s]


In [14]:
# Make data frame from match_dict
match_df = pd.DataFrame(match_dict).T.reset_index()
match_df.columns = ['DISEASE/TRAIT', 'MAPPED_TRAIT_CLEAN', 'TRAIT_CODE_CLEAN']

In [15]:
clean_study_info = study_info.merge(match_df, on='DISEASE/TRAIT', how='left')
print('Updated mappings:', clean_study_info[clean_study_info.MAPPED_TRAIT_URI != clean_study_info.TRAIT_CODE_CLEAN].shape[0])
print('Retained mappings:',clean_study_info[clean_study_info.MAPPED_TRAIT_URI == clean_study_info.TRAIT_CODE_CLEAN].shape[0] )

Updated mappings: 3138
Retained mappings: 16314


In [16]:
clean_study_info.to_csv(os.path.join(datadir,'Reference_Data',  'cleaned_gwas-catalog-v1.0.3.1-studies-r2025-03-26.tsv.update'), sep='\t', index=False)

### CV Association Data Cleaning

Associations are filtered to those:
* With P-value less than a given threshold
* With mapped gene and trait information
* Not in intergenic regions
* Mapped to a single gene and single trait within a single study

In [17]:
def clean_gwas_catalog_data(datafile, outfile, pval_th=5e-8, include_intergenic=False):
    """Clean the GWAS Catalog data and write to a new file.

    Args:
        datafile (str): file path for GWAS Catalog data
        outfile (str): output file for cleaned data
        pval_th (float): p-value threshold for filtering
        include_intergenic (bool): whether to include intergenic associations

    Returns:
        None
    """
    cols= ['DATE', 'PUBMEDID', 'DISEASE/TRAIT', 'MAPPED_GENE', 'SNP_GENE_IDS', 'P-VALUE', 'OR or BETA' ,'MAPPED_TRAIT', 
           'MAPPED_TRAIT_URI', 'INTERGENIC', 'STUDY ACCESSION', 'SNP_ID_CURRENT', 'INITIAL SAMPLE SIZE', 'GENOTYPING TECHNOLOGY']    
    if include_intergenic:
        cols = cols + ['UPSTREAM_GENE_ID', 'DOWNSTREAM_GENE_ID', 'UPSTREAM_GENE_DISTANCE', 'DOWNSTREAM_GENE_DISTANCE']
    data = pd.read_csv(datafile, sep="\t", usecols=cols, low_memory=False)
    # filter on pval
    data = data[data["P-VALUE"] <= pval_th]
    # filter on gene and trait present
    data = data.dropna(subset=['SNP_GENE_IDS', "MAPPED_TRAIT_URI"])
    # filter out intergenic
    if not include_intergenic:
        data = data[data["INTERGENIC"] == 0]
    # remove associations with multiple genes
    data = data[~data["SNP_GENE_IDS"].str.contains(",")]
    # remove associations with multiple traits
    data = data[~data["MAPPED_TRAIT_URI"].str.contains(",")]
    # create trait code
    data['TRAIT_CODE'] = data['MAPPED_TRAIT_URI'].apply(lambda x: x.split('/')[-1])
    # write the cleaned file
    data.to_csv(outfile, sep="\t", index=False)

In [18]:
clean_gwas_catalog_data(os.path.join(datadir, 'Reference_Data', 'gwas_catalog_Jan29_2025.txt.gz'), 
                        os.path.join(datadir, 'Reference_Data', 'gwas_catalog_Jan29_2025.txt.cleaned.update'), pval_th=1)

In [19]:
gwas_genes= pd.read_csv(os.path.join(datadir, 'Reference_Data', 'gwas_catalog_Jan29_2025.txt.cleaned.gz'), sep="\t")

#### Map identifiers to NCBI Gene IDs (Entrez IDs)

In [20]:
# First map from Ensembl
ensembl_map, missing = qe.get_latest_ensembl_id(gwas_genes['SNP_GENE_IDS'].unique())
ensembl_to_entrez, missing_entrez = gm.convert_node_ids(ensembl_map['to'].values, 'Ensembl', 'Entrez')
ensembl_map['Entrez'] = [ensembl_to_entrez[x] if x in ensembl_to_entrez else '' for x in ensembl_map['to']]
id_ensembl = gwas_genes.merge(ensembl_map.loc[:, ('from', 'Entrez')], left_on='SNP_GENE_IDS', right_on='from', how='left')

Query batch 0 - 1000
Query batch 1000 - 2000
Query batch 2000 - 3000
Query batch 3000 - 4000
Query batch 4000 - 5000
Query batch 5000 - 6000
Query batch 6000 - 7000
Query batch 7000 - 8000
Query batch 8000 - 9000
Query batch 9000 - 10000
Query batch 10000 - 11000
Query batch 11000 - 12000
Query batch 12000 - 13000
Query batch 13000 - 14000
Query batch 14000 - 15000
Query batch 15000 - 16000
Query batch 16000 - 17000
Query batch 17000 - 17133


In [22]:
# Try mapping based on symbols for unsuccessful conversions 
symbol_map, symbol_missing = qh.perform_hgnc_query(id_ensembl[(id_ensembl['Entrez'].isnull()) | (id_ensembl['Entrez']== '')]['MAPPED_GENE'].unique(), 'Symbol', 'Symbol')
symbol_to_entrez, missing = gm.convert_node_ids(list(symbol_map.values()), 'Symbol', 'Entrez')
symbol_map = pd.DataFrame(symbol_map.items(), columns=['from', 'to'])
symbol_map['Entrez'] = [symbol_to_entrez[x] if x in symbol_to_entrez else '' for x in symbol_map['to']]
id_symbol = gwas_genes.iloc[~id_ensembl.index].merge(symbol_map.loc[:, ('from', 'Entrez')], left_on='MAPPED_GENE', right_on='from', how='inner')
id_ensembl = id_ensembl[(id_ensembl['Entrez'] != '') & (~id_ensembl['Entrez'].isnull())]

Initial Ids 1108
Checking approved symbols
Response received
Check names 8
Previous Ids 8
Checking previous symbols
Alias Ids 2
Searching aliases
Searching Entrez


In [23]:
# Put all together
converted_gwas_genes = pd.concat([id_ensembl, id_symbol])
converted_gwas_genes = converted_gwas_genes[converted_gwas_genes['Entrez'] != '']
converted_gwas_genes.to_csv(os.path.join(datadir, 'Reference_Data', 'gwas_catalog_Jan29_2025.txt.cleaned.entrez.update'), 
                                sep="\t", index=False)

## RAVAR - Rare Variant Associations

RAVAR Study information can be downloaded from http://www.ravar.bio/#/downloads. Study size and cohort details were manually curated from the associated publications.

### Initial study filtering

Studies are filtered to:
* Exclude reviews and non-genome wide studies
* Exclude studies with missing sample size infomation

In [24]:
# This study information includes manually curated sample size and cohort information
rv_study_info = pd.read_csv(os.path.join(datadir, 'Reference_Data','rv_study_info_cleaned_with_manual.tsv'), sep='\t', index_col=0)
print('Total initial studies:', len(rv_study_info))
# Exclude reviews and other excluded studies
rv_study_info = rv_study_info[(~rv_study_info.COHORT.isin(['Review', 'Exclude'])) &  (rv_study_info['Classification']!='Exclude')]
print('- Excluded studies',  rv_study_info.shape[0])
# exclude studies with missing sample size information
rv_study_info = rv_study_info.dropna(subset=['N'])
print('- Missing N:',  len( rv_study_info))

Total initial studies: 2921
- Excluded studies 2864
- Missing N: 2864


In [25]:
# Load provided trait information
rv_trait_info = pd.read_csv(os.path.join(datadir,'Reference_Data', 'trait_allinfo_06112024.txt'), sep='\t')
print('Total initial traits:', len(rv_trait_info))

Total initial traits: 2005


### Trait mapping

Trait to EFO mappings are updated to harmonize with the GWAS catalog data. 

In [26]:
# expand acronyms in both datasets
rv_study_info['Trait'] = rv_study_info['Reported Trait'].apply(lambda z: z.lower())
rv_study_info['Mapped'] = rv_study_info['Trait Label'].apply(lambda z: z.lower())
rv_study_info['TraitExp'] = rv_study_info['Reported Trait'].apply(lambda x: replace_acronyms(x, all_acronyms).lower())
rv_study_info['MappedExp'] = rv_study_info['Trait Label'].apply(lambda x: replace_acronyms(x, all_acronyms).lower())
rv_trait_info['Mapped'] = rv_trait_info['Trait Label'].apply(lambda x: replace_acronyms(x, all_acronyms).lower())

In [27]:
# Get GWAS mappings (clean and filtered)
clean_study_info = pd.read_csv(os.path.join(datadir, 'Reference_Data', 'cleaned_gwas-catalog-v1.0.3.1-studies-r2025-03-26.tsv.gz'), sep='\t').dropna(subset='MAPPED_TRAIT_CLEAN')
clean_study_info['Trait'] = clean_study_info['DISEASE/TRAIT'].apply(lambda z: z.lower())
clean_study_info['Mapped'] = clean_study_info['MAPPED_TRAIT_CLEAN'].apply(lambda z: z.lower())
# (original)
gwas_trait_info = pd.read_csv(os.path.join(datadir, 'Reference_Data' , 'gwas_catalog_trait-mappings_r2025-03-26.tsv.gz'), sep='\t')
gwas_trait_info['Trait'] = gwas_trait_info['Disease trait'].apply(lambda z: z.lower())
gwas_trait_info['Mapped'] = gwas_trait_info['EFO term'].apply(lambda z: z.lower())
gwas_trait_info['TraitExp'] = gwas_trait_info['Disease trait'].apply(lambda x: replace_acronyms(x, all_acronyms).lower())
gwas_trait_info['MappedExp'] = gwas_trait_info['EFO term'].apply(lambda x: replace_acronyms(x, all_acronyms).lower())
gwas_trait_info = gwas_trait_info[gwas_trait_info['Mapped'].isin(clean_study_info['Mapped'])]

In [28]:
# Identify exact trait description matches between RAVAR and GWAS Catalog
matches = set(gwas_trait_info['Trait']).intersection(set(rv_study_info['Trait']))
print('Exact trait matches:', len(matches))
# add to match dictionary
match_dict = {}
for txt in tqdm(matches):
    match_dict[txt] = map_trait(txt, gwas_trait_info, usecol='Trait')

Exact trait matches: 342


100%|██████████| 342/342 [00:08<00:00, 38.25it/s]


In [29]:
# Identify exact matches between RAVAR traits and Mapped GWAS traits
matches2 = set(gwas_trait_info['Mapped'].values).intersection(set(rv_study_info[~rv_study_info.Trait.isin(matches)]['Trait'].values))
print('Matches to MAPPED GWAS traits:', len(matches2))
# add to trait dictionary
for txt in tqdm(matches2):
     match_dict[txt] = map_trait(txt, gwas_trait_info, usecol='Mapped')

Matches to MAPPED GWAS traits: 138


100%|██████████| 138/138 [00:05<00:00, 24.56it/s]


In [30]:
matched_traits = matches.union(matches2)
print('Total matches so far:', len(matched_traits))

Total matches so far: 480


In [31]:
# Identify matches between GWAS traits and mapped RAVAR traits
matches3 = set(gwas_trait_info['Trait'].values).intersection(set(rv_study_info[~rv_study_info.Trait.isin(matched_traits)]['Mapped'].values))
print('Matches to MAPPED RAVAR traits:',len(matches3))
map_dict = {}
for txt in tqdm(matches3):
     map_dict[txt] = map_trait(txt, gwas_trait_info, usecol='Trait')

Matches to MAPPED RAVAR traits: 193


100%|██████████| 193/193 [00:04<00:00, 38.87it/s]


In [32]:
print('Total matches so far:', len(match_dict) + len(map_dict))

Total matches so far: 673


In [33]:
# Create a final dataset of trait matches
trait_df = pd.DataFrame(match_dict).T.reset_index()
trait_df.columns = ['Trait', 'MAPPED_TRAIT_CLEAN', 'TRAIT_CODE_CLEAN']
trait_df['TRAIT_CODE_CLEAN'] = trait_df['TRAIT_CODE_CLEAN'].apply(lambda x: x.split('/')[-1])
# And trait mappings
mapped_df = pd.DataFrame(map_dict).T.reset_index()
mapped_df.columns = ['Mapped', 'MAPPED_TRAIT_CLEAN', 'TRAIT_CODE_CLEAN']
mapped_df['TRAIT_CODE_CLEAN'] = mapped_df['TRAIT_CODE_CLEAN'].apply(lambda x: x.split('/')[-1])

### Final RV study information

In [34]:
# Combine results
rv_study_info1 = rv_study_info.merge(trait_df, on='Trait', how='inner')
rv_study_info2 = rv_study_info[~rv_study_info.rv_idx.isin(rv_study_info1.rv_idx.values)].merge(mapped_df, on='Mapped', how='inner')
rv_study_info_mapped = pd.concat([rv_study_info1, rv_study_info2])
print('Total mapped:', len(rv_study_info_mapped))

Total mapped: 1165


In [35]:
# For any traits not included about, retain the original mappings
rv_study_info_remaining = rv_study_info[~rv_study_info.rv_idx.isin(rv_study_info_mapped.rv_idx.values)].copy()
rv_study_info_remaining['MAPPED_TRAIT_CLEAN'] = rv_study_info_remaining['Mapped']
rv_study_info_remaining['TRAIT_CODE_CLEAN'] = rv_study_info_remaining['Trait Ontology id'].apply(lambda x: x.replace(':', '_'))
print('Total remaining:', len(rv_study_info_remaining))

Total remaining: 1699


In [36]:
rv_study_info_out = pd.concat([rv_study_info_mapped, rv_study_info_remaining])

In [37]:
rv_study_info_out.to_csv(os.path.join(datadir, 'Reference_Data','rv_study_info_cleaned_with_manual_mapped_Mar28.tsv.update'), sep='\t')

### RV Association Data Cleaning

RV Association Data can be downloaded from http://www.ravar.bio/#/downloads

In [38]:
rv_study_info = pd.read_csv(os.path.join(datadir,'Reference_Data', 'rv_study_info_cleaned_with_manual_mapped_Mar28.tsv'),
                            sep='\t', index_col=0)

In [39]:
ravar_genes = pd.read_csv(os.path.join(datadir,'Reference_Data' ,'gene_fulltable_06112024.txt.gz'),sep='\t', 
                            usecols=['Gene Symbol', 'Ensembl ID', 'Gene Type', 'CHR', 'Location', 'Reported Trait', 
                                     'Trait Label', 'Trait Ontology id', 'EFO synonym', 'P-value', 'PMID'],
                         low_memory=False)
#replace '−' with '-'
ravar_genes['P-value'] = ravar_genes['P-value'].apply(lambda x: float(x.replace('−','-')) if type(x) == str else float(x))
ravar_genes['TRAIT_CODE'] = ravar_genes['Trait Ontology id'].apply(lambda x: x.replace(":", "_") if type(x) == str else x)
ravar_genes['logp'] = -1 * np.log10(ravar_genes['P-value'] + 1e-250)

In [40]:
print('Reported traits', len(ravar_genes['Reported Trait'].unique()))
print('Total traits with Ontology', len(ravar_genes['Trait Ontology id'].unique()))
print('Total genes', len(ravar_genes['Gene Symbol'].unique()))
print('Unique studies', len(ravar_genes['PMID'].unique()))
print('Min p-value', ravar_genes['P-value'].min())
print('Max p-value', ravar_genes['P-value'].max())

Reported traits 3049
Total traits with Ontology 1468
Total genes 12850
Unique studies 200
Min p-value 0.0
Max p-value 9.94e-05


#### Map identifiers to NCBI Gene IDs (Entrez IDs)

In [42]:
ensembl_map, missing = qe.get_latest_ensembl_id(ravar_genes['Ensembl ID'].unique())
ensembl_to_entrez, missing_entrez = gm.convert_node_ids(ensembl_map['to'].values, 'Ensembl', 'Entrez')
ensembl_map['Entrez'] = [ensembl_to_entrez[x] if x in ensembl_to_entrez else '' for x in ensembl_map['to']]
id_ensembl = ravar_genes.merge(ensembl_map.loc[:, ('from', 'Entrez')], left_on='Ensembl ID', right_on='from', how='inner')
id_ensembl = id_ensembl[id_ensembl['Entrez'] != '']

Query batch 0 - 1000
Query batch 1000 - 2000
Query batch 2000 - 3000
Query batch 3000 - 4000
Query batch 4000 - 5000
Query batch 5000 - 6000
Query batch 6000 - 7000
Query batch 7000 - 8000
Query batch 8000 - 9000
Query batch 9000 - 10000
Query batch 10000 - 11000
Query batch 11000 - 12000
Query batch 12000 - 12850


In [43]:
if len(missing_entrez) > 0:
    symbol_map, symbol_missing = qh.perform_hgnc_query(ravar_genes[ravar_genes['Ensembl ID'].isin(missing_entrez)]['Gene Symbol'].unique(), 'Symbol', 'Symbol')
    symbol_to_entrez, missing = gm.convert_node_ids(list(symbol_map.values()), 'Symbol', 'Entrez')
    symbol_map = pd.DataFrame(symbol_map.items(), columns=['from', 'to'])
    symbol_map['Entrez'] = [symbol_to_entrez[x] if x in symbol_to_entrez else '' for x in symbol_map['to']]
    id_symbol = ravar_genes.iloc[~id_ensembl.index].merge(symbol_map.loc[:, ('from', 'Entrez')], left_on='Gene Symbol', right_on='from', how='inner')
    converted_ravar_genes = pd.concat([id_ensembl, id_symbol])
else:
    converted_ravar_genes = id_ensembl

Initial Ids 61
Checking approved symbols
Response received
Check names 4
Previous Ids 2
Checking previous symbols


In [44]:
# add the PCMIDS to the converted data
converted_ravar_genes = converted_ravar_genes.merge(ravar_genes.loc[:, ('PMID', 'Ensembl ID', 'Trait Label', 'P-value', 'Reported Trait', 'Location',
                        'Gene Symbol')].drop_duplicates(), on=['Gene Symbol', 'Ensembl ID', 'Trait Label', 'P-value', 'Reported Trait', 'Location'], how='left')

In [45]:
converted_ravar_genes.to_csv(os.path.join(datadir,'Reference_Data' ,'gene_fulltable_06112024.txt.entrez.update'), sep='\t', index=False)